In [6]:
import psycopg2
import hidden
import ipywidgets as widgets

# Load the secrets
secrets = hidden.secrets()


conn = psycopg2.connect(host=secrets['host'],
        port=secrets['port'],
        database=secrets['database'], 
        user=secrets['user'], 
        password=secrets['pass'], 
        connect_timeout=3)

In [7]:
# defining widgets
cat_widget = widgets.Dropdown(
    options=[('Income', 1), ('Household expenses', 2), ('Fruit', 3), ('Veggies', 4), ('Protein', 5), ('Groceries', 6), ('Relax', 7)],
    description='Category:',
)

date_widget = widgets.DatePicker(
    description='Transaction date:',
    disabled=False
)

detail_widget = widgets.Text(
    placeholder='Insert transaction detail',
    description='Transaction detail:',
    disabled=False
)

amount_widget = widgets.FloatText(
    description='Total amount:',
    disabled=False
)

submit_button = widgets.Button(description='Submit', button_style='success')

In [8]:
# Global list to store rows
pending_rows = []

def on_add_row(button):
    # Get and validate the input
    cat_id = cat_widget.value
    date = date_widget.value.strftime('%Y-%m-%d') if date_widget.value else None
    detail = detail_widget.value
    amount = amount_widget.value

    if not (cat_id and date and detail and amount):
        print('Please fill all the fields')
        return

    row = [cat_id, date, detail, amount]
    pending_rows.append(row)
    print(f"Row added: {row}")

def on_submit_all(button):
    if not pending_rows:
        print("No rows to insert")
        return

    sql = """INSERT INTO transactions (cat_id, tran_date, detail, total_amount) 
             VALUES (%s, %s, %s, %s)"""
    try:
        with conn.cursor() as cur:
            cur.executemany(sql, pending_rows)
        conn.commit()
        print(f"{len(pending_rows)} transaction(s) inserted successfully")
        pending_rows.clear()  # Clear the list after insertion
    except Exception as e:
        conn.rollback()
        print(f'Error inserting transactions: {e}')



In [10]:
# Create two buttons: one for adding a row and one for submitting all rows
add_row_button = widgets.Button(description="Add Row")
submit_all_button = widgets.Button(description="Submit All")

add_row_button.on_click(on_add_row)
submit_all_button.on_click(on_submit_all)

display(cat_widget, date_widget, detail_widget, amount_widget, add_row_button, submit_all_button)

Dropdown(description='Category:', options=(('Income', 1), ('Household expenses', 2), ('Fruit', 3), ('Veggies',…

DatePicker(value=None, description='Transaction date:', step=1)

Text(value='', description='Transaction detail:', placeholder='Insert transaction detail')

FloatText(value=0.0, description='Total amount:')

Button(description='Add Row', style=ButtonStyle())

Button(description='Submit All', style=ButtonStyle())